# Exploring and clustering the neighborhoods in Toronto.

### Importing Libraries

In [74]:
import pandas as pd
import numpy as np
import geocoder
from geopy.geocoders import Nominatim
import folium
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


## Getting Data

In [2]:
# Importing the List of Pincodes
df =pd.read_csv('data.csv')

#Refining the dataframe such that it doesn't contains the Non assigned values
df1 = df[df['Borough']!= 'Not assigned'].reset_index(drop=True)

#Sorting the dataframe according to the Postal codes
df1 = df1.sort_values('Postal Code')

# Importing the list of coordinates for each pincode
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates = coordinates.sort_values('Postal Code')

#Adding the latitude and Longitude column to the df1 dataframe
df1['Latitude'] = coordinates['Latitude']
df1['Longitude'] = coordinates['Longitude']
df1.reset_index(drop=True,inplace= True)
df1.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.709060,-79.363452
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.728020,-79.388790
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.667967,-79.367675
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.650571,-79.384568


In [8]:
# Creating a dataframe that only contains neighbourhoods related to Toronto
df2 = df1[df1['Borough'].str.contains("toronto", case=False)]
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.786947,-79.385975
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.704324,-79.388790
43,M4M,East Toronto,Studio District,43.657162,-79.378937
44,M4N,Central Toronto,Lawrence Park,43.648198,-79.379817


### Getting Coordinates from Toronto 

In [12]:
address = 'Toronto'
geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Creating the map of Neighbourhoods

In [186]:
toronto_map = folium.Map(location=[latitude,longitude],zoom_start=10)
# adding markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#f21312',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### Adding Fourspace API credentials 

In [188]:
CLIENT_ID = 'SECRET' # your Foursquare ID
CLIENT_SECRET = 'SECRET' # your Foursquare Secret
VERSION = '20210128' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


### Creating a dataframe with venues added

In [27]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=df2['Neighbourhood'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [190]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant
1,The Beaches,43.786947,-79.385975,TD Canada Trust,43.788074,-79.380367,Bank
2,The Beaches,43.786947,-79.385975,Maxim's Cafe and Patisserie,43.787863,-79.380751,Café
3,The Beaches,43.786947,-79.385975,Kaga Sushi,43.787758,-79.381090,Japanese Restaurant
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


### Checking Neighborhood Count

In [34]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",4,4,4,4,4,4
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,5,5,5,5,5,5
Christie,2,2,2,2,2,2
Church and Wellesley,7,7,7,7,7,7
"Commerce Court, Victoria Hotel",3,3,3,3,3,3
Davisville,4,4,4,4,4,4
Davisville North,63,63,63,63,63,63


### Total number of Venue Categories in Toronto

In [40]:
toronto_venues['Venue Category'].value_counts().shape

(201,)

# Analysing the data

### Converting categories into numerical values using one hot encoding

In [57]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(0,'Neighbourhood',toronto_venues['Neighborhood'])
toronto_onehot.shape

(749, 202)

### Grouping rows by neighbourhoods and using mean

In [58]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051282,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.025641
1,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.058824
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.063492,0.00,0.047619,0.015873,0.000000,0.00,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000


### Shape of the grouped frame

In [60]:
toronto_grouped.shape

(38, 202)

### Finding the top 5 frequent venues for each neighbourhood

In [64]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0          Coffee Shop  0.08
1            Gastropub  0.05
2              Brewery  0.05
3  American Restaurant  0.05
4                 Café  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
               venue  freq
0               Park  0.25
1     Sandwich Place  0.25
2           Bus Line  0.25
3  Mobile Phone Shop  0.25
4       Liquor Store  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
           venue  freq
0    Yoga Studio  0.06
1  Garden Center  0.06
2    Pizza Place  0.06
3     Comic Shop  0.06
4     Restaurant  0.06


----Central Bay Street----
            venue  freq
0        Pharmacy   0.2
1     Coffee Shop   0.2
2         Butcher   0.2
3  Discount Store   0.2
4     Pizza Place   0.2


----Christie----
               venue  freq
0               Park   0.5
1  Food & Drink

In [65]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Creating the new dataframe and display the top 10 venues for each neighborhood

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood_'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood_,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Gastropub,Brewery,Café,American Restaurant,Bakery,Music Store,Cheese Shop,Pharmacy,Pet Store
1,"Business reply mail Processing Centre, South C...",Mobile Phone Shop,Park,Sandwich Place,Bus Line,Yoga Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run
2,"CN Tower, King and Spadina, Railway Lands, Har...",Yoga Studio,Pizza Place,Restaurant,Comic Shop,Skate Park,Farmers Market,Fast Food Restaurant,Burrito Place,Spa,Butcher
3,Central Bay Street,Coffee Shop,Butcher,Pharmacy,Pizza Place,Discount Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run
4,Christie,Park,Food & Drink Shop,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center


### Clustering the neighbourhoods

In [92]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 3, 1, 0, 1, 1, 1], dtype=int32)

In [93]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [160]:
toronto_merged = df2

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood_'), on='Neighbourhood')


### Venues not returned for on neighbourhood therefore removing it

In [159]:
toronto_merged[toronto_merged['Cluster Labels'].isnull()]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,M5E,Downtown Toronto,Berczy Park,43.75749,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
toronto_merged.drop(axis=0,index=56,inplace = True)

In [175]:
# changing cluster labels to integer
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [192]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [179]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Downtown Toronto,0,Airport,Business Service,Park,Airport Gate,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
61,Downtown Toronto,0,Park,Trail,Lawyer,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
65,Central Toronto,0,Park,Women's Store,Pool,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run
87,East Toronto,0,Mobile Phone Shop,Park,Sandwich Place,Bus Line,Yoga Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run


### Cluster 2

In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,1,Café,Japanese Restaurant,Chinese Restaurant,Bank,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
41,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pizza Place,Lounge,Liquor Store,Juice Bar
42,East Toronto,1,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Gym,Thai Restaurant,Greek Restaurant
43,East Toronto,1,Clothing Store,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Café,Cosmetics Shop,Hotel,Italian Restaurant,Electronics Store
44,Central Toronto,1,Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega,Italian Restaurant
45,Central Toronto,1,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dessert Shop,Caribbean Restaurant,Bar,Grocery Store,Gaming Cafe
46,Central Toronto,1,Field,Trail,Playground,Hockey Arena,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
47,Central Toronto,1,Park,Construction & Landscaping,Basketball Court,Bakery,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
48,Central Toronto,1,Gift Shop,Breakfast Spot,Dog Run,Coffee Shop,Eastern European Restaurant,Cuban Restaurant,Movie Theater,Italian Restaurant,Restaurant,Dessert Shop
49,Central Toronto,1,Coffee Shop,Hotel,Intersection,Fried Chicken Joint,Mediterranean Restaurant,American Restaurant,Burrito Place,Gym,Gas Station,Sandwich Place


### Cluster 3

In [182]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2,Baseball Field,Yoga Studio,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run
70,Downtown Toronto,2,Baseball Field,Yoga Studio,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run


### Cluster 4

In [183]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Downtown Toronto,3,Park,Food & Drink Shop,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center


### Cluster 5

In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Downtown Toronto,4,Gym,Pizza Place,Intersection,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
